<a href="https://colab.research.google.com/github/Smruthi3/END2/blob/main/Session5-Assignment/Session5_Assigment_sentimentanalyis_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Importing the standford data files

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Data overview
Based on Readme file given by StandordSentiment Treebank, the following way sentence and sentiment label are mapped

1. datasetSentences.txt contains sentence and corresponding index
2. datasetSplit contains the sentence index and splitset label. Splitset labels 1,2,3 corresponds to tarin, test and dev respectively
3. datasetSentences and  datasetSplit merged based on sentence index. let's call it as merged_dataset
4. sentiment_labels contains phrase id and sentiment values ranging from 0 to 1. Here notice that there is no one to mapping between datasetSentences and sentiment_labels
5. dictionary.txt contains all phrases and their IDs
6. Now, perfom  left join on merged_dataset and dictionary using sentence and phrases
7. As we have phrase id, again perform left join on previous dataset (obtained from step #6) and sentiment_labels using phrase_id and phrase id
8. Here we get a final table with sentiment values
9. Divide the sentiment values into 5 buckets [0, 0.2], (0.2, 0.4], (0.4, 0.6], (0.6, 0.8], (0.8, 1.0] and assign a label from 0 to 4
10. Finally, use a split label to devide your data into train,test and dev


In [3]:
import os 
import pandas as pd
DATA_DIR = 'drive/My Drive/END2/Session5-Assignment/stanfordSentimentTreebank/stanfordSentimentTreebank'

In [4]:
datasetSentences = pd.read_csv(os.path.join(DATA_DIR, 'datasetSentences.txt'), sep='\t')
datasetSentences.head()

,sentence_index,sentence
0,1,The Rock is destined to be the 21st Century 's...
1,2,The gorgeously elaborate continuation of `` Th...
2,3,Effective but too-tepid biopic
3,4,If you sometimes like to go to the movies to h...
4,5,"Emerges as something rare , an issue movie tha..."


In [5]:
datasetSplit = pd.read_csv(os.path.join(DATA_DIR, 'datasetSplit.txt'), sep=',')
datasetSplit.head()

,sentence_index,splitset_label
0,1,1
1,2,1
2,3,2
3,4,2
4,5,2


In [6]:
datasetSentences.shape, datasetSplit.shape

((11855, 2), (11855, 2))

In [7]:
datasetSentences.columns, datasetSplit.columns

(Index(['sentence_index', 'sentence'], dtype='object'),
 Index(['sentence_index', 'splitset_label'], dtype='object'))

In [8]:
merged_dataset = datasetSentences.merge(datasetSplit, on='sentence_index')
merged_dataset.head()

,sentence_index,sentence,splitset_label
0,1,The Rock is destined to be the 21st Century 's...,1
1,2,The gorgeously elaborate continuation of `` Th...,1
2,3,Effective but too-tepid biopic,2
3,4,If you sometimes like to go to the movies to h...,2
4,5,"Emerges as something rare , an issue movie tha...",2


In [9]:
sentiment_label = pd.read_csv(os.path.join(DATA_DIR, "sentiment_labels.txt"),sep='|')
sentiment_label.head(),sentiment_label.shape

(   phrase ids  sentiment values
 0           0           0.50000
 1           1           0.50000
 2           2           0.44444
 3           3           0.50000
 4           4           0.42708, (239232, 2))

In [10]:
dictionary = pd.read_csv(os.path.join(DATA_DIR, "dictionary.txt"), sep='|',header=None)
dictionary.columns = ['phrases', 'phrase_id']
dictionary.head(),dictionary.shape

(       phrases  phrase_id
 0            !          0
 1          ! '      22935
 2         ! ''      18235
 3       ! Alas     179257
 4  ! Brilliant      22936, (239232, 2))

In [11]:
dataset_with_phrase_id = merged_dataset.merge(dictionary, left_on='sentence', right_on ='phrases', how='left')
dataset_with_phrase_id.head(), dataset_with_phrase_id.shape

(   sentence_index  ... phrase_id
 0               1  ...  226166.0
 1               2  ...  226300.0
 2               3  ...   13995.0
 3               4  ...   14123.0
 4               5  ...   13999.0
 
 [5 rows x 5 columns], (11855, 5))

In [12]:
ads = dataset_with_phrase_id.merge(sentiment_label, left_on='phrase_id', right_on='phrase ids', how='left')

In [13]:
ads['labels'] = [(0 if 0 <=i <=0.2 else (1 if 0.2<i<=0.4 else (2 if 0.4 <i<=0.6 else (3 if 0.6 <i<=0.8 else 4)))) for i in ads['sentiment values'] ]

In [14]:
ads.head()

,sentence_index,sentence,splitset_label,phrases,phrase_id,phrase ids,sentiment values,labels
0,1,The Rock is destined to be the 21st Century 's...,1,The Rock is destined to be the 21st Century 's...,226166.0,226166.0,0.69444,3
1,2,The gorgeously elaborate continuation of `` Th...,1,The gorgeously elaborate continuation of `` Th...,226300.0,226300.0,0.83333,4
2,3,Effective but too-tepid biopic,2,Effective but too-tepid biopic,13995.0,13995.0,0.51389,2
3,4,If you sometimes like to go to the movies to h...,2,If you sometimes like to go to the movies to h...,14123.0,14123.0,0.73611,3
4,5,"Emerges as something rare , an issue movie tha...",2,"Emerges as something rare , an issue movie tha...",13999.0,13999.0,0.86111,4


In [15]:
train_df, test_df, dev_df = [groups for name, groups in ads.groupby('splitset_label')]

In [16]:
ads.shape[0] == train_df.shape[0] + test_df.shape[0] + dev_df.shape[0]

True

In [17]:
train_df=train_df[['sentence','labels']]
train_df=train_df.reset_index(drop=True)
test_df=test_df[['sentence','labels']]
test_df=test_df.reset_index(drop=True)
dev_df=dev_df[['sentence','labels']]
dev_df=dev_df.reset_index(drop=True)

In [18]:
train_df.shape,test_df.shape,dev_df.shape

((8544, 2), (2210, 2), (1101, 2))

In [19]:
train_df.labels.value_counts()

3    2216
1    2092
4    1657
2    1549
0    1030
Name: labels, dtype: int64

### Data Augmentation

### Back Translation

Intially tried for all the sentences in the training data set. It look lot of time but did not convert more than few sentences so for the demonstartion purpose the below mentioned approuch is used

Due to following limitations of the package only a random smaple of 10 sentences (why 10 just randonly chosen) are taken and performed this augmentation. Germen language is chosen here (one can choose multiple langauges too)
  1. The maximum character limit on a single text is 15k.
  2. Due to limitations of the web version of google translate, this API does not guarantee that the library would work properly at all times

If there is time, probably back transaltion can be done on entire training set by passing sentences in multiple batches on different days/time 


In [20]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 61kB 5.6MB/s 
     |████████████████████████████████| 1.0MB 14.4MB/s 
     |████████████████████████████████| 51kB 9.5MB/s 
     |████████████████████████████████| 71kB 9.9MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=5c299c10d4c8c253b1a007344f460e28a3575d75cb7ab0b2099d650f69400488
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [21]:
sample = train_df.sample(10)

In [22]:
import random
import googletrans
 
translator = googletrans.Translator()
sentence = list(sample.sentence)
print(len(sentence))
available_langs = list(googletrans.LANGUAGES.keys()) 
trans_lang = 'de' #random.choice(available_langs) 
print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

translations = translator.translate(sentence, dest=trans_lang) 
#print(translations)
t_text = [t.text for t in translations]
print(t_text)

translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
en_text = [t.text for t in translations_en_random]
print(en_text)

10
Translating to german
['Es ist wahrscheinlich, dass Sie, was immer Sie von der ersten Produktion dachten – für oder gegen –, wahrscheinlich an diese denken werden.', 'Ein gelungener Film von selbstreflexiver, philosophischer Natur.', 'Ver Wiels verzweifelter Versuch des Witzes geht verloren und der Charakter von Critical Jim wird zweidimensional und sinnlos.', 'Erwachsene Spitzfindigkeiten gehören hier nicht zur Sache.', 'Queen of the Damned zu sehen ist wie das Lesen einer Forschungsarbeit mit Spezialeffekten.', 'Eine lächerliche – oder eher unlächerliche – Ausrede für einen Film.', 'Kritiker brauchen auch ein gutes Lachen, und diese für das Fernsehen zu extreme Wiedergabe der berüchtigten MTV-Show liefert die unverschämten, widerlichen, abstoßenden Waren in dampfenden, viszeralen Haufen.', 'Der Anblick des Raumschiffs auf der Startrampe ist in IMAX-Dimensionen gebührend beeindruckend, ebenso wie Aufnahmen der in ihren Kabinen schwebenden Astronauten.', 'Der Film hat die unheimlich

In [23]:
d = {'sentence':en_text,'labels':list(sample.labels)}
d1=pd.DataFrame(d)
d1.shape


(10, 2)

In [24]:
train_df_agumented = pd.concat([train_df,d1])
train_df_agumented = train_df_agumented.reset_index(drop=True)
train_df_agumented.shape

(8554, 2)

### Radom swap

In [25]:
def random_swap(sentence, n=5): 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return sentence

In [26]:
data_rand_swap=pd.DataFrame()
data_rand_swap['sentence']=train_df['sentence'].apply(lambda x : " ".join(random_swap(x.split(' '))))
data_rand_swap['labels'] = train_df['labels'] 

In [27]:
train_df_agumented1=pd.concat([train_df_agumented,data_rand_swap])
train_df_agumented1=train_df_agumented1.reset_index(drop=True)
train_df_agumented1.shape

(17098, 2)

### Random Deletion

In [28]:
def random_deletion(words, p=0.5): 
    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return [random.choice(words)] 
    else:
        return remaining

In [29]:
data_rand_del=pd.DataFrame()
data_rand_del['sentence'] = train_df['sentence'].apply(lambda x : " ".join(random_deletion(x.split(' '))))
data_rand_del['labels'] = train_df['labels'] 

In [30]:
train_df_agumented2=pd.concat([train_df_agumented1,data_rand_del])
train_df_agumented2=train_df_agumented2.reset_index(drop=True)
train_df_agumented2.shape

(25642, 2)

In [31]:
#train_df_agumented2.to_csv("drive/My Drive/END2/Session5-Assignment/train_df_agumented2.csv",index=False)

### Defining Data Fields

In [32]:
# Import Library
import random
import torch, torchtext
from torchtext.legacy import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [33]:
Review = data.Field(sequential = True , tokenize ='spacy',batch_first = True , include_lengths =True)
Label = data.LabelField(tokenize ='spacy',is_target=True,batch_first=True,sequential=False)

In [34]:
fields = [('reviews', Review),('labels',Label)]

In [35]:
train_df_agumented2.shape

(25642, 2)

In [36]:
train_example = [data.Example.fromlist([train_df_agumented2.sentence[i],train_df_agumented2.labels[i]], fields) for i in range(train_df_agumented2.shape[0])] 
test_example = [data.Example.fromlist([test_df.sentence[i],test_df.labels[i]], fields) for i in range(test_df.shape[0])] 
valid_example = [data.Example.fromlist([dev_df.sentence[i],dev_df.labels[i]],fields) for i in range(dev_df.shape[0])]

In [37]:
train = data.Dataset(train_example, fields)
valid = data.Dataset(valid_example, fields)
test = data.Dataset(test_example, fields)

In [38]:
(len(train), len(test),len(valid))

(25642, 2210, 1101)

In [39]:
vars(train.examples[10])

{'labels': 4,
 'reviews': ['Good',
  'fun',
  ',',
  'good',
  'action',
  ',',
  'good',
  'acting',
  ',',
  'good',
  'dialogue',
  ',',
  'good',
  'pace',
  ',',
  'good',
  'cinematography',
  '.']}

#### Building Vocabulary

In [40]:
MAX_VOCAB_SIZE = 25_000

Review.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

Label.build_vocab(train)

.vector_cache/glove.6B.zip: 862MB [02:39, 5.39MB/s]                           
 99%|█████████▉| 397880/400000 [00:13<00:00, 28931.20it/s]

In [41]:
print('Size of input vocab : ', len(Review.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Review .vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  17214
Size of label vocab :  5
Top 10 words appreared repeatedly : [('.', 20111), (',', 17797), ('the', 15203), ('of', 11208), ('and', 11188), ('a', 11018), ('to', 7486), ('-', 6855), ('is', 6341), ("'s", 6323)]
Labels :  defaultdict(None, {3: 0, 1: 1, 4: 2, 2: 3, 0: 4})


In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [43]:
batch_size=64
train_iterator , valid_iterator = data.BucketIterator.splits((train,valid),batch_size=batch_size,sort_key= lambda x: len(x.reviews),
                                                             sort_within_batch=True,device=device)
train_iterator , test_iterator = data.BucketIterator.splits((train,test),batch_size=batch_size,sort_key= lambda x: len(x.reviews),
                                                             sort_within_batch=True,device=device)

In [44]:
import os, pickle
with open('drive/My Drive/END2/Session5-Assignment/Model_artifacts/tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Review.vocab.stoi, tokens)

### Defining the Model

In [45]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout, pad_idx, bidirectional=False):
        
        super().__init__()          
        
        self.bidirectional = bidirectional
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           bidirectional=bidirectional,
                           batch_first=True)

        
        # Dense layer
        self.dropout = nn.Dropout(dropout)
        scaling_factor = 2 if bidirectional else 1
        
        self.fc = nn.Linear(hidden_dim*scaling_factor, output_dim)

    def forward(self, text, text_lengths):
      
        # text = [batch size, sent_length]
        embedded = self.dropout(self.embedding(text))
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        if self.bidirectional:
          hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
          hidden = self.dropout(hidden)
          hidden = hidden[0]
          
        dense_outputs = self.fc(hidden)   
        # Final activation function softmax
        output = F.softmax(dense_outputs, dim=1)
            
        return output

In [46]:
# Define hyperparameters
size_of_vocab = len(Review.vocab)
embedding_dim = 300
num_hidden_nodes = 256
num_output_nodes = 5
num_layers = 2
dropout = 0.5
bidirectional=True
PAD_IDX = Review.vocab.stoi[Review.pad_token]


# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout, pad_idx=PAD_IDX, bidirectional=bidirectional)

In [47]:
UNK_IDX = Review.vocab.stoi[Review.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(embedding_dim)
model.embedding.weight.data[PAD_IDX] = torch.zeros(embedding_dim)

In [48]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(17214, 300, padding_idx=1)
  (encoder): LSTM(300, 256, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=5, bias=True)
)
The model has 7,886,509 trainable parameters


#### Model Training and Evaluation

In [49]:
import torch.optim as optim

# define optimizer and loss
# lr=0.005
# lr = 2e-4
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()



# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc

def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    _, predictions = torch.max(preds, 1)
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

Training Loop

In [50]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        review, review_lengths = batch.reviews   
        
        # convert to 1D tensor
        predictions = model(review, review_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        #acc = binary_accuracy(predictions, batch.labels)   
        acc = categorical_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Evaluation Loop

In [51]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            review, review_lengths = batch.reviews 
            
            # convert to 1d tensor
            predictions = model(review, review_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            # acc = binary_accuracy(predictions, batch.labels)
            acc = categorical_accuracy(predictions, batch.labels)   
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Model Training and Evaluation 

In [52]:
N_EPOCHS = 30
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'drive/My Drive/END2/Session5-Assignment/Model_artifacts/saved_weights.pt')
    
    print(f'\t Epoch: {epoch} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Epoch: {epoch} | Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	 Epoch: 0 | Train Loss: 1.559 | Train Acc: 31.07%
	 Epoch: 0 | Val. Loss: 1.510 |  Val. Acc: 37.03% 

	 Epoch: 1 | Train Loss: 1.505 | Train Acc: 37.86%
	 Epoch: 1 | Val. Loss: 1.505 |  Val. Acc: 38.24% 

	 Epoch: 2 | Train Loss: 1.451 | Train Acc: 44.02%
	 Epoch: 2 | Val. Loss: 1.498 |  Val. Acc: 37.81% 

	 Epoch: 3 | Train Loss: 1.415 | Train Acc: 47.74%
	 Epoch: 3 | Val. Loss: 1.496 |  Val. Acc: 39.38% 

	 Epoch: 4 | Train Loss: 1.378 | Train Acc: 51.71%
	 Epoch: 4 | Val. Loss: 1.482 |  Val. Acc: 40.15% 

	 Epoch: 5 | Train Loss: 1.346 | Train Acc: 55.10%
	 Epoch: 5 | Val. Loss: 1.489 |  Val. Acc: 40.33% 

	 Epoch: 6 | Train Loss: 1.333 | Train Acc: 56.37%
	 Epoch: 6 | Val. Loss: 1.497 |  Val. Acc: 38.94% 

	 Epoch: 7 | Train Loss: 1.312 | Train Acc: 58.66%
	 Epoch: 7 | Val. Loss: 1.496 |  Val. Acc: 39.64% 

	 Epoch: 8 | Train Loss: 1.291 | Train Acc: 60.69%
	 Epoch: 8 | Val. Loss: 1.497 |  Val. Acc: 40.32% 

	 Epoch: 9 | Train Loss: 1.273 | Train Acc: 62.59%
	 Epoch: 9 | Val. Loss

In [54]:
model.load_state_dict(torch.load('drive/My Drive/END2/Session5-Assignment/Model_artifacts/saved_weights.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.493 | Test Acc: 40.66%


### Validation of the model by passing the reviews and observing it's outcome

In [55]:
#load weights and tokenizer

path = 'drive/My Drive/END2/Session5-Assignment/Model_artifacts/saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('drive/My Drive/END2/Session5-Assignment/Model_artifacts/tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_review(review):
    # labels: very negative, negative, neutral, positive, very positive
    # defaultdict(None, {3: 0, 1: 1, 4: 2, 2: 3, 0: 4})
    categories = {4: "very negative", 1:"negative", 3:"neutral", 0:'positive', 2: 'very positive'}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(review)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [56]:
test_sample_reviews= test_df.sample(10).reset_index(drop=True)

In [57]:
actual_lable= {0: "very negative", 1:"negative", 2:"neutral", 3:'positive', 4: 'very positive'}

for i in range(test_sample_reviews.shape[0]):
  print(f'Review : {test_sample_reviews.sentence[i]}')
  print(f'Actual Sentiment : {actual_lable[test_sample_reviews.labels[i]]} | Predicted Sentiment : {classify_review(test_sample_reviews.sentence[i])} \n')


Review : The film 's bathos often overwhelms what could have been a more multifaceted look at this interesting time and place .
Actual Sentiment : neutral | Predicted Sentiment : very negative 

Review : What happens to John Q ?
Actual Sentiment : neutral | Predicted Sentiment : positive 

Review : The movie has lots of dancing and fabulous music .
Actual Sentiment : very positive | Predicted Sentiment : positive 

Review : This is a throwaway , junk-food movie whose rap soundtrack was better tended to than the film itself .
Actual Sentiment : very negative | Predicted Sentiment : negative 

Review : You might not buy the ideas .
Actual Sentiment : negative | Predicted Sentiment : negative 

Review : The movie addresses a hungry need for PG-rated , nonthreatening family movies , but it does n't go too much further .
Actual Sentiment : neutral | Predicted Sentiment : neutral 

Review : Director Dirk Shafer and co-writer Greg Hinton ride the dubious divide where gay porn reaches for seri